In [272]:
from scipy import misc
import tensorflow as tf
import numpy as np
import imageio



In [266]:
def extract_image_patches(image, patch_dims, step):
    patches = []
    for ii in range(0,image.shape[0],step[0]):
        for jj in range(0,image.shape[1],step[1]):
            patch = (image[ii:(ii+patch_dims[0]),jj:(jj+patch_dims[1])]).tolist()
            patches.append(patch)
            
    return np.asarray(patches)

def combine_image_patches(patches, patch_dims, step, image_dims):
    image = np.empty(image_dims, dtype=int)
    cnt = 0
    for ii in range(0,image.shape[0],step[0]):
        for jj in range(0,image.shape[1],step[1]):
            image[ii:(ii+patch_dims[0]),jj:(jj+patch_dims[1])] = np.maximum(
                np.minimum(np.rint(patches[cnt]),256),0)
            cnt = cnt+1
            
    return image

In [294]:

patch_dims = [16,16]
image_dims = [512,512]
input_image= imageio.imread('./lena_gray.bmp', flatten= 0)
patches = extract_image_patches(input_image, patch_dims, patch_dims)
patches_reshaped = patches.reshape(int(image_dims[0]/patch_dims[0]*image_dims[1]/patch_dims[1]),
                                   patch_dims[0]*patch_dims[1])/256


In [297]:

input_units = patch_dims[0]*patch_dims[1]
hidden_units = 128
display_step = 100
X = tf.placeholder("float", [None, input_units])
weights = {
    'encoder':tf.Variable(tf.random_normal([input_units, hidden_units])),
    'decoder':tf.Variable(tf.random_normal([hidden_units, input_units]))
}

biases = {
    'encoder':tf.Variable(tf.random_normal([hidden_units])),
    'decoder':tf.Variable(tf.random_normal([input_units]))
}

def encoder(x):
    layer = tf.nn.softmax(tf.add(tf.matmul(x,weights['encoder']),biases['encoder']))
    #layer = tf.add(tf.matmul(x,weights['encoder']),biases['encoder'])
    return layer
                          
def decoder(x):
    layer = tf.nn.sigmoid(tf.add(tf.matmul(x,weights['decoder']),biases['decoder']))
    #layer = tf.add(tf.matmul(x,weights['decoder']),biases['decoder'])
    return layer  

In [298]:

batch_size = 64
lr = 0.01
num_steps = 10000
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)
y_pred = decoder_op
y_true = X

loss = tf.reduce_mean(tf.pow(y_pred - y_true, 2))
#optimiser = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(loss)
optimiser = tf.train.MomentumOptimizer(learning_rate=lr,momentum =0.99).minimize(loss)
init = tf.global_variables_initializer()


with tf.Session() as sess:
    sess.run(init)
    
    for ii in range(1,num_steps+1):
        for jj in range(0,(len(patches_reshaped)-batch_size),batch_size):
            end = jj+batch_size
            batch = patches_reshaped[jj:end,:]
            
            _, l = sess.run([optimiser, loss], feed_dict={X:batch})

        if ii % display_step == 0 or ii == 1:
            print('Step %i: Minibatch Loss: %f' % (ii, l))

            


    g = sess.run([decoder_op], feed_dict={X:patches_reshaped})
    

patches_new = g[0].reshape(int(image_dims[0]/patch_dims[0]*image_dims[1]/patch_dims[1]),
                                   patch_dims[0],
                                   patch_dims[1])*256
image = combine_image_patches(patches_new, patch_dims, patch_dims,image_dims)
imageio.imsave("rec.png",im=image)

Step 1: Minibatch Loss: 0.098973
Step 100: Minibatch Loss: 0.065135
Step 200: Minibatch Loss: 0.049576
Step 300: Minibatch Loss: 0.042308
Step 400: Minibatch Loss: 0.039208
Step 500: Minibatch Loss: 0.037575
Step 600: Minibatch Loss: 0.036324
Step 700: Minibatch Loss: 0.035065
Step 800: Minibatch Loss: 0.033664
Step 900: Minibatch Loss: 0.032122
Step 1000: Minibatch Loss: 0.030487
Step 1100: Minibatch Loss: 0.028829
Step 1200: Minibatch Loss: 0.027198
Step 1300: Minibatch Loss: 0.025620
Step 1400: Minibatch Loss: 0.024116
Step 1500: Minibatch Loss: 0.022700
Step 1600: Minibatch Loss: 0.021377
Step 1700: Minibatch Loss: 0.020153
Step 1800: Minibatch Loss: 0.019029
Step 1900: Minibatch Loss: 0.018003
Step 2000: Minibatch Loss: 0.017072
Step 2100: Minibatch Loss: 0.016232
Step 2200: Minibatch Loss: 0.015478
Step 2300: Minibatch Loss: 0.014804
Step 2400: Minibatch Loss: 0.014206
Step 2500: Minibatch Loss: 0.013676
Step 2600: Minibatch Loss: 0.013209
Step 2700: Minibatch Loss: 0.012796
Step

/home/am2442/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from int64 to uint8, range [30, 204]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))


In [291]:
patches_reshaped

array([[ 0.625     ,  0.625     ,  0.625     , ...,  0.6171875 ,
         0.60546875,  0.61328125],
       [ 0.6328125 ,  0.62109375,  0.625     , ...,  0.6015625 ,
         0.59375   ,  0.59375   ],
       [ 0.609375  ,  0.60546875,  0.625     , ...,  0.6015625 ,
         0.58984375,  0.59765625],
       ..., 
       [ 0.33203125,  0.32421875,  0.30078125, ...,  0.19921875,
         0.19921875,  0.22265625],
       [ 0.22265625,  0.21875   ,  0.234375  , ...,  0.2890625 ,
         0.30078125,  0.328125  ],
       [ 0.2421875 ,  0.2265625 ,  0.25390625, ...,  0.41796875,
         0.421875  ,  0.4375    ]])